In [1]:
bucket = 'eduthie-sagemaker-1'
prefix = 'gluon_recommender'

import sagemaker
role = sagemaker.get_execution_role()

import os
import mxnet as mx
from mxnet import gluon, nd, ndarray
from mxnet.metric import MSE
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.mxnet import MXNet
import boto3
import json
import matplotlib.pyplot as plt

In [ ]:
opt = 'sgd'
lr = 0.02
momentum = 0.9
wd = 0.

m = MXNet('recommender.py', 
          py_version='py3',
          role=role, 
          train_instance_count=10, 
          train_instance_type="ml.p2.16xlarge",
          output_path='s3://{}/{}/output'.format(bucket, prefix),
          hyperparameters={'num_embeddings': 512, 
                           'opt': opt, 
                           'lr': lr, 
                           'momentum': momentum, 
                           'wd': wd,
                           'epochs': 10})

m.fit({'train': 's3://{}/{}/train/'.format(bucket, prefix)})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-06-04-08-35-45-986


........................................
2018-06-04 08:42:13,336 INFO - root - running container entrypoint
2018-06-04 08:42:13,337 INFO - root - starting train task
2018-06-04 08:42:13,710 INFO - container_support.training - Training starting
2018-06-04 08:42:15,171 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'available_cpus': 64, '_scheduler_ip': '10.32.0.4', 'hyperparameters': {'sagemaker_enable_cloudwatch_metrics': False, 'sagemaker_program': 'recommender.py', 'sagemaker_submit_directory': 's3://sagemaker-eu-west-1-987551451182/sagemaker-mxnet-2018-06-04-08-35-45-986/source/sourcedir.tar.gz', 'sagemaker_region': 'eu-west-1', 'num_embeddings': 512, 'sagemaker_container_log_level': 20, 'lr': 0.02, 'wd': 0.0, 'opt': 'sgd', 'sagemaker_job_name': 'sagemaker-mxnet-2018-06-04-08-35-45-986', 'epochs': 10, 'momentum': 0.9}, '_scheduler_host': 'algo-1', 'output_dir': '/opt/ml/output', 'user_script_name': 'recommender.py', 'user_script_archive': 's3://sagemaker-eu-west-1-9875514

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/sagemaker/session.py:630: DeprecationWarning: generator 'multi_stream_iter' raised StopIteration
  for idx, event in sagemaker.logs.multi_stream_iter(client, log_group, stream_names, positions):


epoch: 0
EPOCH 0: MSE ON TRAINING and TEST: 0.9683504440737865. 1.3518057554076717
epoch: 1
EPOCH 1: MSE ON TRAINING and TEST: 0.8440807554593421. 1.2407326508173242
epoch: 2
EPOCH 2: MSE ON TRAINING and TEST: 0.79589849029699. 1.205945230196384
epoch: 3
EPOCH 3: MSE ON TRAINING and TEST: 0.8308787389937806. 1.2394235321366256
epoch: 4
EPOCH 7: MSE ON TRAINING and TEST: 0.6722131938437601. 1.219422062555224
epoch: 8
EPOCH 8: MSE ON TRAINING and TEST: 0.5154444976276445. 1.2505068283702248
epoch: 9
EPOCH 9: MSE ON TRAINING and TEST: 0.3398741641003822. 1.2781291879297254
end of training
===== Job Complete =====
Billable seconds: 8941


### Calculate total cost of training

The cost of training on such a large instance with 16 GPUs is relatively expensive at 21.773 euros per hour. Fortunately it is charged by the second. Here we calculate the total price.

There is a lot more that can be done to optimise training speed and hence cost. 

In [5]:
price_per_second = 21.773/(60*60)
total_price = 8941*price_per_second
print(total_price)

54.07566472222222


## Host

Now that we've trained our model, deploying it to a real-time, production endpoint is easy.

In [6]:
predictor = m.deploy(initial_instance_count=1, 
                     instance_type='ml.m4.xlarge')
predictor.serializer = None

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-06-04-08-35-45-986
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-06-04-08-35-45-986


---------------------------------------------------------------!

Now that we have an endpoint, let's test it out. We'll predict user #6's ratings for the top and bottom ASINs from our local model.

This could be done by sending HTTP POST requests from a separate web service, but to keep things easy, we'll just use the .predict() method from the SageMaker Python SDK.

In [12]:
predictor.predict(json.dumps({'customer_id': '6', 
                              'product_id': ['B00KH1O9HW', 'B00M5KODWO']}))

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/sagemaker-mxnet-2018-06-04-08-35-45-986 in account 987551451182 for more information.

Note, some of our predictions are actually greater than 5, which is to be expected as we didn't do anything special to account for ratings being capped at that value. Since we are only looking to ranking by predicted rating, this won't create problems for our specific use case.

### Evaluate

Let's start by calculating a naive baseline to approximate how well our model is doing.  The simplest estimate would be to assume every user item rating is just the average rating over all ratings.

*Note, we could do better by using each individual video's average, however, in this case it doesn't really matter as the same conclusions would hold.*

In [ ]:
print('Naive MSE:', np.mean((test_df['star_rating'] - np.mean(train_df['star_rating'])) ** 2))

Now, we'll calculate predictions for our test dataset.

*Note, this will align closely to our CloudWatch output above, but may differ slightly due to skipping partial mini-batches in our eval_net function.*

In [ ]:
test_preds = []
for array in np.array_split(test_df[['customer_id', 'product_id']].values, 40):
    test_preds += predictor.predict(json.dumps({'customer_id': array[:, 0].tolist(), 
                                                'product_id': array[:, 1].tolist()}))

test_preds = np.array(test_preds)
print('MSE:', np.mean((test_df['star_rating'] - test_preds) ** 2))

We can see that our neural network and embedding model produces substantially better results (~1.27 vs 1.65 on mean square error).

For recommender systems, subjective accuracy also matters.  Let's get some recommendations for a random user to see if they make intuitive sense.

In [ ]:
reduced_df[reduced_df['user'] == 6].sort_values(['star_rating', 'item'], ascending=[False, True])

As we can see, user #6 seems to like sprawling dramamtic television series and sci-fi, but they dislike silly comedies.

Now we'll loop through and predict user #6's ratings for every common video in the catalog, to see which ones we'd recommend and which ones we wouldn't.

In [ ]:
predictions = []
for array in np.array_split(product_index['product_id'].values, 40):
    predictions += predictor.predict(json.dumps({'customer_id': customer_index[customer_index['user'] == 6]['customer_id'].values.tolist() * array.shape[0], 
                                                 'product_id': array.tolist()}))

predictions = pd.DataFrame({'product_id': product_index['product_id'],
                            'prediction': predictions})

In [ ]:
titles = reduced_df.groupby('product_id')['product_title'].last().reset_index()
predictions_titles = predictions.merge(titles)
predictions_titles.sort_values(['prediction', 'product_id'], ascending=[False, True])

Indeed, our predicted highly rated shows have some well-reviewed TV dramas and some sci-fi. Meanwhile, our bottom rated shows include goofball comedies.

Note, because of random initialization in the weights, results on subsequent runs may differ slightly.

Let's confirm that we no longer have almost perfect correlation in recommendations with user #7.

In [ ]:
predictions_user7 = []
for array in np.array_split(product_index['product_id'].values, 40):
    predictions_user7 += predictor.predict(json.dumps({'customer_id': customer_index[customer_index['user'] == 7]['customer_id'].values.tolist() * array.shape[0], 
                                                       'product_id': array.tolist()}))
plt.scatter(predictions['prediction'], np.array(predictions_user7))
plt.show()

---

## Wrap-up

In this example, we developed a deep learning model to predict customer ratings.  This could serve as the foundation of a recommender system in a variety of use cases.  However, there are many ways in which it could be improved.  For example we did very little with:
- hyperparameter tuning
- controlling for overfitting (early stopping, dropout, etc.)
- testing whether binarizing our target variable would improve results
- including other information sources (video genres, historical ratings, time of review)
- adjusting our threshold for user and item inclusion 

In addition to improving the model, we could improve the engineering by:
- Setting the context and key value store up for distributed training
- Fine tuning our data ingestion (e.g. num_workers on our data iterators) to ensure we're fully utilizing our GPU
- Thinking about how pre-processing would need to change as datasets scale beyond a single machine

Beyond that, recommenders are a very active area of research and techniques from active learning, reinforcement learning, segmentation, ensembling, and more should be investigated to deliver well-rounded recommendations.

### Clean-up (optional)

Let's finish by deleting our endpoint to avoid stray hosting charges.

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)